# Select Your IPTS 

In [ ]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Description 

# Python Import

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Import Counts vs Time files 

Select the file created by the [profile_vs_file_index.ipynb](profile_vs_file_index.ipynb) notebook

In [ ]:
import ipywe.fileselector

from ipywidgets import widgets


In [ ]:
list_files_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select Counts vs Time Files ...',
                                                 start_dir= system.System.get_working_dir(),
                                                 multiple=True)

list_files_ui.show()

# Import Data and Metadata 

In [ ]:
def retrieve_metadata(file_name):
    """
    return a dictionary of the metadata (plus number of comment lines)
    """
    with open(file_name) as f:
        _metadata = {}
        _nbr_comment_line = 0
        for line in f:
            li = line.strip()
            if li.startswith("#"):
                _nbr_comment_line += 1
                m = re.match(r"^#(?P<tag_name>[,\(\)\s\w]*):(?P<tag_value>.*)", li)
                if m is None:
                    continue
                _metadata[m.group('tag_name')] = m.group('tag_value').strip()
        _metadata['nbr_comment_line'] = _nbr_comment_line
        return _metadata
    
def retrieve_data(file_name, nbr_comment_line=0, col_names = None):
    file_0 = pd.read_csv(file_name, 
                         skiprows = nbr_comment_line, 
                         header = None,
                         names = col_names)
    return file_0

def format_col_names(col_names):
    _col_names = [_col.strip() for _col in col_names]
    return _col_names

In [ ]:
list_files = list_files_ui.selected

In [ ]:
import re

In [ ]:
w = widgets.IntProgress()
w.max = len(list_files)
display(w)
index = 0

files_data = {}
files_metadata = {}
for index, file in enumerate(list_files):
    _metadata = retrieve_metadata(file)
    files_metadata[file] = _metadata
    _col_names = _metadata['Label'].split(',')
    _new_col_names = format_col_names(_col_names)
    _data = retrieve_data(file, 
                          nbr_comment_line = _metadata['nbr_comment_line'],
                          col_names = _new_col_names)
    files_data[file] = _data

    index += 1
    w.value = index

# Calculation of Water Intake Peak 

In [ ]:
import numpy as np

In [ ]:
class MeanRangeCalculation(object):
    '''
    Mean value of all the counts between left_pixel and right pixel
    '''
    
    def __init__(self, data=None):
        self.data = data
        self.nbr_pixel = len(self.data)
        
    def calculate_left_right_mean(self, pixel=-1):
        _data = self.data
        _nbr_pixel = self.nbr_pixel
        
        self.left_mean = np.mean(_data[0:pixel+1])
        self.right_mean = np.mean(_data[pixel+1:_nbr_pixel])
        
    def calculate_delta_mean_square(self):
        self.delta_square = np.square(self.left_mean - self.right_mean)

In [ ]:
pixel_index = files_data[list_files[0]]
nbr_pixel = len(pixel_index)

water_intake_peak = []
time_stamps = []
for _index, _file in enumerate(list_files):
        counts = files_data[_file]['counts'].values
        delta_array = []
        for _pixel in range(0, nbr_pixel-1):
            _o_range = MeanRangeCalculation(data=counts)
            _o_range.calculate_left_right_mean(pixel = _pixel)
            _o_range.calculate_delta_mean_square()
            delta_array.append(_o_range.delta_square)

        time_stamps.append(files_metadata[_file]['Delta Time (s)'])
            
        _peak_value = delta_array.index(max(delta_array[0:nbr_pixel-5]))
        water_intake_peak.append(_peak_value)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(time_stamps, water_intake_peak)
plt.xlabel("Delta Time (s)")
plt.ylabel("Peak Position (pixel)")
plt.title("Peak vs Delta Time (s)")

In [ ]:
from ipywidgets import widgets, Layout


In [ ]:
box = widgets.HBox([widgets.Label("Size/Pixel Ratio"),
                   widgets.FloatText(value=1,
                                     step=0.1,
                                    layout=widgets.Layout(width='10%')),
                   widgets.Label("mm/px")])
                   
display(box)

In [ ]:
ratio_widget = box.children[1]

In [ ]:
water_intake_peak_px = np.array(water_intake_peak)

In [ ]:
water_intake_peak_mm = water_intake_peak_px * ratio_widget.value

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot(time_stamps, water_intake_peak_mm)
plt.xlabel("Delta Time (s)")
plt.ylabel("Peak Position (mm)")
plt.title("Peak vs Delta Time (s)")

# Output

In [ ]:
output_folder_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select Output Folder ...',
                                                             start_dir=system.System.get_working_dir(),
                                                             type='directory')
output_folder_ui.show()


In [ ]:
output_folder = output_folder_ui.selected

In [ ]:
short_file_name = os.path.basename(os.path.dirname(list_files[0])) + '.txt'
output_file_name = os.path.join(output_folder, short_file_name)

ascii_text = "# Source data: {}\n".format(os.path.basename(os.path.dirname(list_files[0])))
ascii_text += '# time_stamp(s), water_intake_position (mm)\n'
for _index in range(len(time_stamp)):
    _x_value = time_stamp[_index]
    _y_value = water_intake_peak_mm[_index]
    ascii_text += "{}, {}\n".format(_x_value, _y_value)
    
f = open(output_file_name, 'w')
f.write(ascii_text)
f.close()